In [5]:
import os
import sys
import json
import pickle
from fpdf import FPDF

# Add the parent directory to the Python path to load funtions from file ML_funtions
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

# Import helperfunctions
from ML_functions import fun_load_settings

# Define the optimization problem (choose either "TSP" or "CVRP")
default_optimization_problem = "TSP"

# Call the function to define optimization_problem based on how the notebook is executed
# If the notebook is run by the script "main.ipynb", load optimization_problem from "settings.json". Otherwise use the default optimization problem from above
optimization_problem = fun_load_settings(default_optimization_problem)

###############################################################################
# DEFINE MODEL NAMES AND FILE NAMES
###############################################################################

# Create a dictionary with all model names as keys and their corresponding names in a file as values
model_names = {"K-nearest Neighbor (KNN)": "_KNN_GS", 
               "Ridge Regression": "_Ridge_GS", 
               "Decision Tree": "_DT_RGS", 
               "Random Forest": "_RF_RGS", 
               "Gradient Boosting Regression Tree": "_GBRT_RGS", 
               "Extreme Gradient Boosting (XGBOOST)": "_XGBOOST_GS", 
               "Linear Support Vector Machine (SVM)": "_LSVM_GS", 
               "Gaussian Kernel Machine": "_SVM_GK_GS", 
               "Multilayer Perceptron Neural Network": "_NN_GS"}

# Exclude the models that were not tuned for the CVRP problem
if (optimization_problem == "CVRP"):
    for model_name in ["Ridge Regression", "Decision Tree", "Gradient Boosting Regression Tree", "Linear Support Vector Machine (SVM)"]:
        del model_names[model_name]

# Create another dictionary with the three different file categories
file_names = {"Parameter grid/distribution": "_param_grid.pkl",
              "Best parameters": "_best_params.pkl", 
              "Tuning details": "_tuning_details.json"}

###############################################################################
# DEFINE FUNCTION to add tuning results to a pdf
###############################################################################

# Function to load all three tuning files for a given model and add the content of each files to the pdf
def fun_pdf_tuning_results(model_key, model_names, file_names, optimization_problem, pdf):
    # Iterate over the three tuning files
    for i, file_key in enumerate(file_names, start=1):

        # Add the dictionary key of the file as subtitle to the pdf
        pdf.set_font("Arial", style="b", size=12)
        pdf.cell(0, 3, '', ln=True) # Add some space on top
        pdf.cell(4, 6, ln=False) # Add some space on the left
        pdf.cell(200, 6, txt=f"  {i}. {file_key}", ln=True, align="left")
        pdf.set_font("Arial", size=12)
        
        # Combine the file name with the subfolder to get the file path
        file_name = optimization_problem + model_names[model_key] + file_names[file_key]
        subfolder = "../03_tuning_results"
        file_path = os.path.join(subfolder, file_name)

        try: 
            # Load the file, depending on whether it is in Pickle or Json format
            if (".pkl" in file_name):
                with open(file_path, "rb") as file:
                    data = pickle.load(file)

                # Get the keys of the parameter grid/distribution dictionary and reorder the dictionary with the best combination
                if (i == 1): keys = data.keys()
                else: data = {key: data[key] for key in keys}

            else:
                with open(file_path, "r") as file:
                    data = json.load(file)
            
            # Add the content of the file to the pdf
            for param in data:
                pdf.cell(12, 6, ln=False) # Add some space on the left
                pdf.cell(200, 6, txt=f"- {param}: {data[param]}", ln=True, align="left")
        
        except FileNotFoundError:
            pdf.cell(12, 6, ln=False) # Add some space on the left
            pdf.cell(200, 6, txt="Error: File not found :(", ln=True)
    
    # Add some space above the next model
    pdf.cell(0, 6, "", ln=True)

###############################################################################
# CREATE THE PDF AND APPLY fun_pdf_tuning_results
###############################################################################
# Create a PDF object, add a page, set the font and create a headline
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", style="b", size=24)
pdf.cell(200, 20, txt=f"Tuning results for the {optimization_problem}", ln=True, align="C")

# Iterate over all models, display the model name as headline and execute the defined function from above for each model
for model_key in model_names:
    pdf.set_font("Arial", style="b", size=18)
    pdf.cell(200, 6, txt=f"{model_key}", ln=True, align="left")
    fun_pdf_tuning_results(model_key, model_names, file_names, optimization_problem, pdf)

# Save the PDF file
pdf.output(f"b_{optimization_problem}_tuning_results.pdf")
print("File saved succesfully.")

The notebook is executed directly. :)
Optimization Problem: 'TSP'
File saved succesfully.
